In [1]:
pip install -r requirements.txt


  Using cached torch-2.8.0-cp310-cp310-manylinux_2_28_x86_64.whl.metadata (30 kB)
  Using cached torchvision-0.23.0-cp310-cp310-manylinux_2_28_x86_64.whl.metadata (6.1 kB)
  Using cached scikit_image-0.25.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (14 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached fsspec-2025.9.0-py3-none-any.whl.metadata (10 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.8.93-py3-none-manylinux2010_x86_64.manylinux_2_12_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_cuda_runtime_cu12-12.8.90-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_cuda_cupti_cu12-12.8.90-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_cudnn_cu12-9.10.2.21-py3-none-manylinux_2_27_x86_64.whl.metadata (1.8 kB)
  Usin

In [2]:
import sys
!{sys.executable} -m pip install -r requirements.txt


In [3]:
import torch
print("Python executable:", sys.executable)
print("Torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("CUDA device count:", torch.cuda.device_count())
    print("Current device index:", torch.cuda.current_device())
    print("Device name:", torch.cuda.get_device_name(torch.cuda.current_device()))


Python executable: /home/shubhamkn22/.conda/envs/cnn_env/bin/python3.10
Torch version: 2.8.0+cu128
CUDA available: True
CUDA device count: 2
Current device index: 0
Device name: NVIDIA A40


In [4]:
import os, sys
PROJECT_ROOT = os.path.abspath('.')  # assuming you're in /gpu_work/cgs401
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)
print("Project root added:", PROJECT_ROOT)


Project root added: /home/shubhamkn22/gpu_work/cgs401


In [6]:
!mkdir -p data/raw data/processed


In [5]:
!pip install opencv-python


In [6]:
import cv2
print("OpenCV version:", cv2.__version__)


OpenCV version: 4.12.0


In [10]:
!pip install cupy-cuda12x opencv-python tqdm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 MB 33.7 MB/s  0:00:03m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [cupy-cuda12x] [cupy-cuda12x]


In [11]:
!python tools/build_maps_from_fixations.py \
  --ann data/raw/annotations/fixations_train2014.json \
  --images data/raw/train/train/ \
  --out data/raw/train/maps/ \
  --sigma 15


Loading annotation json: data/raw/annotations/fixations_train2014.json
Building maps: 100%|██████████████████████| 10000/10000 [17:05<00:00,  9.75it/s]
Done. Processed 10000/10000 images. Missing images: 0


In [12]:
!python tools/build_maps_from_fixations.py \
  --ann data/raw/annotations/fixations_val2014.json \
  --images data/raw/val/val/ \
  --out data/raw/val/maps/ \
  --sigma 15


Loading annotation json: data/raw/annotations/fixations_val2014.json
Building maps: 100%|████████████████████████| 5000/5000 [09:39<00:00,  8.63it/s]
Done. Processed 5000/5000 images. Missing images: 0


In [8]:
!ls -R data/raw/annotations


data/raw/annotations:
fixations_train2014.json  fixations_val2014.json


In [13]:
!python scripts/prepare_salicon.py \
  --dataset SALICON \
  --raw_dir data/raw \
  --out_dir data/processed \
  --img_size 256


 CUDA available: False

 Processing split: train
Found 10000 images for train split.
Resizing train: 100%|████████████████████| 10000/10000 [01:17<00:00, 128.71it/s]

 Processing split: val
Found 0 images for val split.
Resizing val: 0it [00:00, ?it/s]
 Built index.json with 10000 train and 0 val samples.

 Preprocessing complete. Output saved to: data/processed


In [23]:
!python train.py \
  --data_root data/processed \
  --epochs 20 \
  --batch_size 16 \
  --lr 2e-4 \
  --out_dir runs/salicon


 Using 2 GPUs via DataParallel
/home/shubhamkn22/gpu_work/cgs401/train.py:129: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(device.type == 'cuda' and not args.no_amp))
 Starting training on cuda for 20 epochs

Epoch 1/20
 Epoch 1: train_loss=0.2101 val_loss=0.1810 AUC-Judd=0.978 NSS=4.721            
 Saved new best model (NSS=4.721)

Epoch 2/20
 Epoch 2: train_loss=0.1684 val_loss=0.1753 AUC-Judd=0.980 NSS=4.341            

Epoch 3/20
 Epoch 3: train_loss=0.1556 val_loss=0.1699 AUC-Judd=0.981 NSS=4.516            

Epoch 4/20
 Epoch 4: train_loss=0.1450 val_loss=0.1685 AUC-Judd=0.981 NSS=4.928            
 Saved new best model (NSS=4.928)

Epoch 5/20
 Epoch 5: train_loss=0.1365 val_loss=0.1665 AUC-Judd=0.982 NSS=5.020            
 Saved new best model (NSS=5.020)

Epoch 6/20
 Epoch 6: train_loss=0.1289 val_loss=0.1694 AUC-Judd=0.981 NSS=5.108            
 Sa

In [24]:
!python eval.py \
  --data_root data/processed \
  --ckpt runs/salicon/best.pt \
  --batch_size 32 \
  --num_workers 8


 Using 2 GPUs for evaluation via DataParallel
 Running evaluation on cuda with batch_size=32, workers=8
Evaluating: 100%|███████████████████████████████████████████████████| 32/32 [00:17<00:00,  1.79it/s]

 Evaluation Complete:
val_loss = 0.169446
AUC-Judd = 0.981107
NSS       = 5.098165


In [17]:
!ls -R data/processed | head -n 30


data/processed:
index.json
train
val

data/processed/train:
fixations
images
maps

data/processed/train/fixations:

data/processed/train/images:
COCO_train2014_000000000009.jpg
COCO_train2014_000000000089.jpg
COCO_train2014_000000000110.jpg
COCO_train2014_000000000307.jpg
COCO_train2014_000000000321.jpg
COCO_train2014_000000000332.jpg
COCO_train2014_000000000349.jpg
COCO_train2014_000000000382.jpg
COCO_train2014_000000000389.jpg
COCO_train2014_000000000419.jpg
COCO_train2014_000000000443.jpg
COCO_train2014_000000000450.jpg
COCO_train2014_000000000508.jpg
COCO_train2014_000000000510.jpg
COCO_train2014_000000000531.jpg
COCO_train2014_000000000532.jpg
COCO_train2014_000000000605.jpg
ls: write error: Broken pipe


In [18]:
!ls -R data/processed/val | head -n 30


data/processed/val:
fixations
images
maps

data/processed/val/fixations:

data/processed/val/images:

data/processed/val/maps:


In [19]:
# Run in notebook cell from project root
import random, shutil
from pathlib import Path
import json

random.seed(42)
ROOT = Path("data/processed")
train_img_dir = ROOT / "train" / "images"
train_map_dir = ROOT / "train" / "maps"
train_fix_dir = ROOT / "train" / "fixations"

val_img_dir = ROOT / "val" / "images"
val_map_dir = ROOT / "val" / "maps"
val_fix_dir = ROOT / "val" / "fixations"

# create val dirs if missing
for d in (val_img_dir, val_map_dir, val_fix_dir):
    d.mkdir(parents=True, exist_ok=True)

# list all train image stems
imgs = sorted([p for p in train_img_dir.glob("*.jpg")])
N = len(imgs)
if N == 0:
    raise SystemExit("No images found in data/processed/train/images — aborting.")

ratio = 0.10  # 10% -> change if you want
k = max(1, int(round(N * ratio)))
print(f"Found {N} train images; moving {k} ({ratio*100:.1f}%) to validation.")

candidates = [p.stem for p in imgs]
sel = random.sample(candidates, k)

moved = 0
for stem in sel:
    src_img = train_img_dir / f"{stem}.jpg"
    src_map = train_map_dir / f"{stem}.png"
    src_fix_j = train_fix_dir / f"{stem}.jpg"
    src_fix_p = train_fix_dir / f"{stem}.png"
    dst_img = val_img_dir / f"{stem}.jpg"
    dst_map = val_map_dir / f"{stem}.png"
    dst_fix = val_fix_dir / f"{stem}.png"

    # move image
    if src_img.exists():
        shutil.move(str(src_img), str(dst_img))
    else:
        # try common alternatives
        print("Warning: image missing", src_img)

    # move map if exists
    if src_map.exists():
        shutil.move(str(src_map), str(dst_map))
    else:
        print("Warning: map missing for", stem)

    # move fixation if present (try png or jpg or mat won't be moved)
    if src_fix_p.exists():
        shutil.move(str(src_fix_p), str(dst_fix))
    elif src_fix_j.exists():
        # convert jpg -> png name or just move
        shutil.move(str(src_fix_j), str(val_fix_dir / f"{stem}.jpg"))
    # else no fixation present; ok

    moved += 1

print(f"Moved {moved} samples to data/processed/val/")

# Rebuild index.json
train_list = sorted([p.stem for p in (ROOT/'train'/'images').glob("*.jpg")])
val_list = sorted([p.stem for p in (ROOT/'val'/'images').glob("*.jpg")])
backup = ROOT / "index.json.bak"
if (ROOT / "index.json").exists():
    shutil.copy2(str(ROOT/"index.json"), str(backup))
    print("Backed up existing index.json ->", backup)

with open(ROOT / "index.json", "w") as f:
    json.dump({"train": train_list, "val": val_list}, f)
print("Wrote new index.json (train len:", len(train_list), "val len:", len(val_list), ")")


Found 10000 train images; moving 1000 (10.0%) to validation.
Moved 1000 samples to data/processed/val/
Backed up existing index.json -> data/processed/index.json.bak
Wrote new index.json (train len: 9000 val len: 1000 )


In [20]:
# verify counts
!echo "Train images:" $(ls data/processed/train/images | wc -l)
!echo "Val images:" $(ls data/processed/val/images | wc -l)
!head -n 10 data/processed/index.json


Train images: 9000
Val images: 1000
{"train": ["COCO_train2014_000000000009", "COCO_train2014_000000000089", "COCO_train2014_000000000110", "COCO_train2014_000000000307", "COCO_train2014_000000000321", "COCO_train2014_000000000332", "COCO_train2014_000000000382", "COCO_train2014_000000000389", "COCO_train2014_000000000443", "COCO_train2014_000000000450", "COCO_train2014_000000000508", "COCO_train2014_000000000510", "COCO_train2014_000000000531", "COCO_train2014_000000000532", "COCO_train2014_000000000605", "COCO_train2014_000000000716", "COCO_train2014_000000000853", "COCO_train2014_000000000965", "COCO_train2014_000000001011", "COCO_train2014_000000001014", "COCO_train2014_000000001144", "COCO_train2014_000000001145", "COCO_train2014_000000001261", "COCO_train2014_000000001264", "COCO_train2014_000000001386", "COCO_train2014_000000001403", "COCO_train2014_000000001424", "COCO_train2014_000000001510", "COCO_train2014_000000001518", "COCO_train2014_000000001523", "COCO_train2014_0000000